In [1]:
import pandas as pd
import numpy as np
import sqlite3
from Query_Executor import QueryExecutor as qe

           TeamName
0     Bayern Munich
1          Dortmund
2        Leverkusen
3        RB Leipzig
4        Schalke 04
5        M'gladbach
6         Wolfsburg
7           FC Koln
8        Hoffenheim
9            Hertha
10            Mainz
11          Hamburg
12    Werder Bremen
13    Ein Frankfurt
14         Augsburg
15         Freiburg
16        Stuttgart
17         Hannover
18    Bayern Munich
19         Dortmund
20       Leverkusen
21       Schalke 04
22        Wolfsburg
23       M'gladbach
24          Hamburg
25            Mainz
26          FC Koln
27       Hoffenheim
28           Hertha
29    Werder Bremen
..              ...
438  Greuther Furth
439         CZ Jena
440     Munich 1860
441        RW Essen
442      Burghausen
443  Erzgebirge Aue
444       Offenbach
445       Paderborn
446    Unterhaching
447    Braunschweig
448        Augsburg
449         Koblenz
450     Munich 1860
451          Bochum
452   Hansa Rostock
453        Freiburg
454          Aachen
455         Dresden


In [2]:
df = pd.read_csv('matches_adding_rain_info.csv')
#df['rain_game'] = df['rain_game'].astype(int)
df

,Match_ID,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,rain_game
0,1092,2012-03-31,Nurnberg,Bayern Munich,0,1,A,0
1,1093,2011-12-11,Stuttgart,Bayern Munich,1,2,A,0
2,1094,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,0
3,1095,2011-11-27,Mainz,Bayern Munich,3,2,H,0
4,1096,2012-02-18,Freiburg,Bayern Munich,0,0,D,0
5,1097,2012-01-20,M'gladbach,Bayern Munich,3,1,H,0
6,1098,2012-02-04,Hamburg,Bayern Munich,1,1,D,0
7,1099,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,0
8,1100,2011-09-18,Schalke 04,Bayern Munich,0,2,A,1
9,1101,2011-10-23,Hannover,Bayern Munich,2,1,H,0


In [4]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()
df.to_sql('with_rain', conn, if_exists='append', index=False)
conn.commit()

In [20]:
q3 = """ SELECT HomeTeam as Club,
            (h_goals_for + a_goals_for) AS total_goals,
            (h_win + a_win) AS wins,
            (h_rain_win + h_rain_draw + h_rain_loss + a_rain_win + a_rain_draw + a_rain_loss) AS rain_games,
            (h_rain_win + a_rain_win) AS rain_wins
            
            FROM 
            
            (SELECT HomeTeam,
                SUM(CASE WHEN FTHG > FTAG THEN 1 ELSE 0 END) AS h_win,
                SUM(CASE WHEN FTHG = FTAG THEN 1 ELSE 0 END) AS h_draw,
                SUM(CASE WHEN FTHG < FTAG THEN 1 ELSE 0 END) AS h_loss,
                SUM(FTHG) AS h_goals_for,
                SUM(FTAG) AS h_goals_agst,
                SUM(CASE WHEN FTHG > FTAG AND rain_game = 1 THEN 1 ELSE 0 END) AS h_rain_win,
                SUM(CASE WHEN FTHG = FTAG AND rain_game = 1 THEN 1 ELSE 0 END) AS h_rain_draw,
                SUM(CASE WHEN FTHG < FTAG AND rain_game = 1 THEN 1 ELSE 0 END) AS h_rain_loss
                
                FROM with_rain
                
                GROUP BY HomeTeam
                ORDER BY HomeTeam)

            JOIN

            (SELECT AwayTeam,
                SUM(CASE WHEN FTAG > FTHG THEN 1 ELSE 0 END) AS a_win,
                SUM(CASE WHEN FTAG = FTHG THEN 1 ELSE 0 END) AS a_draw,
                SUM(CASE WHEN FTAG < FTHG THEN 1 ELSE 0 END) AS a_loss,
                SUM(FTAG) AS a_goals_for,
                SUM(FTHG) AS a_goals_agst,
                SUM(CASE WHEN FTAG > FTHG AND rain_game = 1 THEN 1 ELSE 0 END) AS a_rain_win,
                SUM(CASE WHEN FTAG = FTHG AND rain_game = 1 THEN 1 ELSE 0 END) AS a_rain_draw,
                SUM(CASE WHEN FTAG < FTHG AND rain_game = 1 THEN 1 ELSE 0 END) AS a_rain_loss            
            
                FROM with_rain
                
                GROUP BY AwayTeam
                ORDER BY AwayTeam)
            
            ON (HomeTeam == AwayTeam)
            
            ORDER BY total_goals DESC, wins DESC, rain_games DESC, rain_wins DESC;
            """

c.execute(q3)
print(c.fetchall())

[('Man City', 186, 56, 10, 6), ('Man United', 178, 56, 8, 8), ('Dortmund', 160, 50, 8, 2), ('Bayern Munich', 154, 46, 8, 8), ('Ein Frankfurt', 152, 40, 4, 2), ('Arsenal', 148, 42, 6, 2), ('Schalke 04', 148, 40, 2, 0), ('Greuther Furth', 146, 40, 8, 4), ('Tottenham', 132, 40, 8, 8), ('Chelsea', 130, 36, 12, 8), ('Fortuna Dusseldorf', 128, 32, 6, 2), ('Stuttgart', 126, 30, 2, 0), ('Munich 1860', 124, 34, 6, 6), ('St Pauli', 118, 36, 2, 2), ('Newcastle', 112, 38, 2, 0), ('Union Berlin', 110, 28, 4, 2), ('Leverkusen', 104, 30, 6, 4), ('Norwich', 104, 24, 6, 2), ('Paderborn', 102, 34, 8, 6), ('Everton', 100, 30, 2, 2), ('Dresden', 100, 24, 2, 0), ("M'gladbach", 98, 34, 4, 2), ('Werder Bremen', 98, 22, 6, 2), ('Fulham', 96, 28, 4, 2), ('Blackburn', 96, 16, 6, 2), ('Liverpool', 94, 28, 8, 4), ('Wolfsburg', 94, 26, 4, 0), ('Mainz', 94, 18, 6, 0), ('Bolton', 92, 20, 4, 0), ('West Brom', 90, 26, 6, 0), ('Sunderland', 90, 22, 8, 4), ('Freiburg', 90, 20, 4, 2), ('Swansea', 88, 24, 6, 2), ('QPR', 8

In [ ]:
#/ (h_rain_win + h_rain_draw + h_rain_loss + a_rain_win + a_rain_draw + a_rain_loss)